In [9]:
from datasets import load_dataset
# DATASET_NAME='arc_easy'
# dataset = load_dataset("allenai/ai2_arc", "ARC-Easy", split=["train", "test"])
DATASET_NAME='arc_hard'
dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge", split=["train", "test"])

In [10]:
def get_prompt(dataset, index, has_choice=False):
    try:
        json_line = dataset[index]
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        perms=list(range(len(choice_texts)))
        choice_texts = [choice_texts[perms[i]] for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        answer_key = json_line["answerKey"][0]
        answer_key_idx = ord(answer_key) - (ord("A") if answer_key in 'ABCDE' else ord("1"))
        answer_text = choices["text"][answer_key_idx]
        fact=f"{json_line['fact1']}. " if 'fact1' in json_line else ''
        prompt = f"{fact}{question} \\n {candidates}"
        return prompt, answer_text
    except:
        print(answer_key)

In [11]:
from tqdm import tqdm
import pickle

for index, dataname in enumerate(['train','test']):
    container=[]
    for ques_index, ques in enumerate(tqdm(dataset[index])):
        container.append(
                get_prompt(dataset[index], ques_index, has_choice=False)
            )
    pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 1172/1172 [00:00<00:00, 9821.18it/s]
